In [1]:
import re
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

import warnings
# ignore some deprecation warnings
warnings.filterwarnings('ignore')

# Load main data set

In [2]:
df = pd.read_csv('retrosynthesis-all', header=None)
df['source'] = df[0].apply(lambda x: x.split('>>')[0])
df['target'] = df[0].apply(lambda x: x.split('>>')[1])
df.drop(0, axis=1, inplace=True)
df.head(3)

,source,target
0,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc...
1,Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1,COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1
2,CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.FC(F)(F)...


# SMILES Vocabulary: How is it generated?

The model's Vocabulary handles the transformation of SMILES strings into a sequence of tokens. Tokens are the pre-defined lowest and indivisible unit of string text. In Natural Language Processing (NLP), tokens are typically defined on the word or character level. The level of tokenization dictates *what* the model can output, e.g., if tokenization on the character level is used, then the model outputs individual characters.

For generative SMILES models, tokenization is performed on the character level where each token *loosely* maps to a unique atom type (brackets, "(" for example indicate branching and thus, do not map to an atom but rather gives connectivity information).


In [3]:
import sys
sys.path.append('src/')
from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, Vocabulary, create_vocabulary

tk = SMILESTokenizer()
vocabulary = Vocabulary()

#### Tokenization example

In [4]:
# smi_sample = df['source'].iloc[123]
smi_sample = 'CCBr'
print(tk.tokenize(smi_sample, with_begin_and_end=False))

['C', 'C', 'Br']


Much like in a natural language like english where one's vocabulary controls what sentences can be formulated, a molecular generative model's vocabulary controls what kind of atoms can be proposed - sentences in this context are molecules

In order for the token representations of SMILES sequences to be passed into a machine learning model, they must be transformed into a numerical representation. This is done in the Vocabulary class where each unique token is mapped to a numerical index. This is shown below:


In [5]:
# create a vocabulary using all SMILES in df
smiles_dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
smiles_dataset = np.unique(smiles_dataset).tolist()

vocabulary = create_vocabulary(smiles_list=smiles_dataset, tokenizer=tk)
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'The unique tokens are: \n{vocabulary.tokens()}')

There are 86 unique tokens in the vocabulary.

The unique tokens are: 
['$', '^', ' ', '#', '(', ')', '-', '.', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[B-]', '[BH-]', '[BH3-]', '[Br-]', '[C-]', '[C@@H]', '[C@@]', '[C@H]', '[C@]', '[Cl+3]', '[Cl-]', '[Cu]', '[Fe]', '[I+]', '[K]', '[Li]', '[Mg+]', '[Mg]', '[N+]', '[N-]', '[N@+]', '[NH+]', '[NH-]', '[NH2+]', '[NH3+]', '[NH4+]', '[O-]', '[OH-]', '[P+]', '[PH2]', '[PH4]', '[PH]', '[Pd]', '[Pt]', '[S+]', '[S-]', '[S@@]', '[S@]', '[SH]', '[Se]', '[SiH2]', '[SiH]', '[Si]', '[SnH]', '[Sn]', '[Zn+]', '[Zn]', '[n+]', '[n-]', '[nH]', '[s+]', '[se]', '\\', 'c', 'n', 'o', 's']


In [6]:
print(smi_sample)
tokenized_smi_sample = tk.tokenize(smi_sample, with_begin_and_end=False)
print(tokenized_smi_sample)

vocabulary.encode(tokenized_smi_sample)

CCBr
['C', 'C', 'Br']


array([21., 21., 20.], dtype=float32)

Τhe `encode` method in the `Vocabulary` class takes a list of tokens and returns its numerical indices the indices are what we expect

# RNN section

This section describes *how* the numerical representation of tokens are transformed into an input vector known as the embedding that will act as the input to the RNN.

An Embedding Layer is essentially a look-up table. In the constructor above, `num_embeddings` refers to the Vocabulary size. 

`num_embeddings` denotes how many vectors to initialize. 

Since we have n unique tokens, we need _ different vectors: 1 for each unique token. This is why `num_embeddings` is _ in this example. `embedding_dim` denotes the dimension of the embedding vector. 5 is arbitrarily chosen here just for easy visualization.



In [7]:
# construct an "Embedding layer"
EMBEDDING_DIM = 5
NUM_EMBEDDING = len(vocabulary)

embedding_layer = nn.Embedding(num_embeddings=NUM_EMBEDDING,
                               embedding_dim=EMBEDDING_DIM)

# only 1 layer of LSTM cells is initialized here for the sake of illustration
# input_size = 5 because we previously defined the "embedding_dim" of the Embedding layer to be 5
# hidden_size = 5 is arbitrarily chosen for easy visualization
recurrent_layer = nn.LSTM(input_size=EMBEDDING_DIM,
                          hidden_size=5,
                          num_layers=1,
                          dropout=0,
                          batch_first=True)

In [8]:
# get the numerical indices of bromoethane
numerical_indices_smi_sample = torch.LongTensor([vocabulary.encode(tokenized_smi_sample).astype(int)])
print(f"Numerical indices of bromoethane:\n {numerical_indices_smi_sample}\n")

embedding = embedding_layer(numerical_indices_smi_sample)
print(f"Embedding:\n {embedding}")

Numerical indices of bromoethane:
 tensor([[21, 21, 20]])

Embedding:
 tensor([[[-0.3277,  0.9600, -0.4915, -0.2581, -1.1825],
         [-0.3277,  0.9600, -0.4915, -0.2581, -1.1825],
         [ 1.1409, -0.3243, -1.1002,  1.4983, -0.0269]]],
       grad_fn=<EmbeddingBackward0>)


In [9]:
print(embedding.shape)

# let's run the embedding through the recurrent layer
rnn_output, (hidden_state, cell_state) = recurrent_layer(embedding)

print(rnn_output.shape)

torch.Size([1, 3, 5])
torch.Size([1, 3, 5])


In [10]:
# initialize the linear layer
# in_features = 5 as that is the hidden_size defined in the recurrent layer above
# out_features = 20 as that is the size of the Vocabulary
linear_layer = nn.Linear(in_features=5,
                         out_features=NUM_EMBEDDING)

linear_output = linear_layer(rnn_output)

# verify the shape of the linear output is what we expect:
# (batch size) x (sequence length) x (vocabulary size)
print(linear_output.shape)

# let's first show the normal softmax output
# recall the output from the linear layer has dimensions: (batch size) x (sequence length) x (vocabulary size)
# therefore, dim=2 because we want to compute the softmax over the vocabulary to obtain a probability for each token
softmax = linear_output.softmax(dim=2)
print(f"Softmax shape: {softmax.shape}")

# let's now show the log-softmax output
log_softmax = linear_output.log_softmax(dim=2)
print(f"Log-Softmax shape: {log_softmax.shape}")

# log-softmax to token probabilities
# recall our original SMILES 
print(f"Original SMILES string: {smi_sample}\n")

# recall our vocabulary
print(f"The unique tokens are: \n{vocabulary.tokens()}\n")

# we now extract the max value in each tensor of the log-softmax output above and the corresponding token
most_probable_tokens = log_softmax.argmax(dim=2).flatten().tolist()
for idx, (correct_token, most_probable_token) in enumerate(zip(smi_sample, most_probable_tokens)):
    print(f"At time step {idx+1}, the generative model proposes {vocabulary.tokens()[most_probable_token]} as the most probable token and the correct token is {correct_token}")

torch.Size([1, 3, 86])
Softmax shape: torch.Size([1, 3, 86])
Log-Softmax shape: torch.Size([1, 3, 86])
Original SMILES string: CCBr

The unique tokens are: 
['$', '^', ' ', '#', '(', ')', '-', '.', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[B-]', '[BH-]', '[BH3-]', '[Br-]', '[C-]', '[C@@H]', '[C@@]', '[C@H]', '[C@]', '[Cl+3]', '[Cl-]', '[Cu]', '[Fe]', '[I+]', '[K]', '[Li]', '[Mg+]', '[Mg]', '[N+]', '[N-]', '[N@+]', '[NH+]', '[NH-]', '[NH2+]', '[NH3+]', '[NH4+]', '[O-]', '[OH-]', '[P+]', '[PH2]', '[PH4]', '[PH]', '[Pd]', '[Pt]', '[S+]', '[S-]', '[S@@]', '[S@]', '[SH]', '[Se]', '[SiH2]', '[SiH]', '[Si]', '[SnH]', '[Sn]', '[Zn+]', '[Zn]', '[n+]', '[n-]', '[nH]', '[s+]', '[se]', '\\', 'c', 'n', 'o', 's']

At time step 1, the generative model proposes [n+] as the most probable token and the correct token is C
At time step 2, the generative model proposes [Cu] as the most probable token and the correct token is C
At time step 

# How Does The Model Learn?

In [11]:
# define the loss
loss = torch.nn.NLLLoss(reduction='none')
# recall the correct token indices of "CCBr" which was our original SMILES
print(f"These are the token indices we would want our model to predict:\n{numerical_indices_smi_sample}\n")
print(f"Recall the log-softmax output:\n{log_softmax.shape}\n")
print(f"We will transpose the log-softmax tensor to have shape \
(batch size) x (vocabulary) x (sequence length):\n{log_softmax.transpose(1,2).shape}\n")

print(f"The output tensor from negative log-likelihood is:\n{loss(log_softmax.transpose(1, 2), numerical_indices_smi_sample)}\n")

These are the token indices we would want our model to predict:
tensor([[21, 21, 20]])

Recall the log-softmax output:
torch.Size([1, 3, 86])

We will transpose the log-softmax tensor to have shape (batch size) x (vocabulary) x (sequence length):
torch.Size([1, 86, 3])

The output tensor from negative log-likelihood is:
tensor([[4.1875, 4.1391, 4.5312]], grad_fn=<ViewBackward0>)



During backpropagation, we take the sum of the `Negative Log-Likehood` tensor to "summarize" the total loss associated with a given SMILES sequence:

In [12]:
loss(log_softmax.transpose(1, 2), numerical_indices_smi_sample).sum(dim=1)

tensor([13.0889], grad_fn=<SumBackward1>)

# Model Training

In [13]:
class Dataset(torch.utils.data.Dataset):
    """
    Custom PyTorch Dataset that takes a file containing \n separated SMILES.
    """

    def __init__(self, smiles_list : list, vocabulary : Vocabulary,
                 tokenizer : SMILESTokenizer) -> None:
        self._vocabulary  = vocabulary
        self._tokenizer   = tokenizer
        self._smiles_list = list(smiles_list)

    def __getitem__(self, i : int) -> torch.Tensor:
        smi     = self._smiles_list[i]
        tokens  = self._tokenizer.tokenize(smi)
        encoded = self._vocabulary.encode(tokens)
        return torch.tensor(encoded, dtype=torch.long)  # pylint: disable=E1102

    def __len__(self) -> int:
        return len(self._smiles_list)

    @staticmethod
    def collate_fn(encoded_seqs : list) -> torch.Tensor:
        """
        Converts a list of encoded sequences into a padded tensor.
        """
        max_length   = max([seq.size(0) for seq in encoded_seqs])
        collated_arr = torch.zeros(len(encoded_seqs),
                                   max_length,
                                   dtype=torch.long)  # padded with zeros
        for i, seq in enumerate(encoded_seqs):
            collated_arr[i, :seq.size(0)] = seq
        return collated_arr

In [14]:
benzene = 'c1ccccc'
# tokenize
benzene_tokenized = tk.tokenize(benzene)
# encode (obtain numerical token indices)
benzene_encoded = vocabulary.encode(benzene_tokenized)
# transform it into a tensor for collating
benzene_encoded = torch.tensor([benzene_encoded])
# collate
benzene_collated = Dataset.collate_fn(benzene_encoded)

In [166]:
from typing import List, Tuple, Union

class SmilesLSTM(torch.nn.Module):
    def __init__(self, vocabulary: Vocabulary, tokenizer: SMILESTokenizer,
                model_parameters: Union[dict, None]=None) -> None:
        
        self.vocabulary = vocabulary
        self.tokenizer = tokenizer
        self.max_sequence_length = len(self.vocabulary)
        
        self._layer_size           = model_parameters['layer_size']
        self._embedding_layer_size = model_parameters['embedding_layer_size']
        self._num_layers           = model_parameters['num_layers']
        self._cell_type            = model_parameters['cell_type'].lower()
        self._dropout              = model_parameters['dropout']
        self._layer_normalization  = model_parameters['layer_normalization']
        
        super(SmilesLSTM, self).__init__()
        # define model
        self._embedding = nn.Embedding(num_embeddings=len(self.vocabulary),
                               embedding_dim=self._embedding_layer_size)
        
        self._reccurent = nn.LSTM(input_size=len(self.vocabulary),
                          hidden_size=self._layer_size,
                          num_layers=self._num_layers,
                          dropout=self._dropout,
                          batch_first=True)
        
        self._linear = nn.Linear(self._layer_size, len(self.vocabulary))
        
    def forward(self, x):

        embedded_data = self._embedding(x) # (batch,seq,embedding)

        output_vector, hidden_state_out, cell_state = self._reccurent(embedded_data)

        output_vector = output_vector.reshape(-1, self._layer_size)

        output_data = self._linear(output_vector).view(batch_size, seq_size, -1)

        return output_data
    
    def get_params(self) -> dict:
        """
        Returns the configuration parameters of the model.
        """
        return {
            "dropout"              : self._dropout,
            "layer_size"           : self._layer_size,
            "num_layers"           : self._num_layers,
            "cell_type"            : self._cell_type,
            "embedding_layer_size" : self._embedding_layer_size
        }

In [167]:
# define network parameters
model_parameters = {
    'num_layers'          : 3,
    'layer_size'          : 5,
    'cell_type'           : 'lstm',
    'embedding_layer_size': 5,
    'dropout'             : 0.2,
    'layer_normalization' : True,
}

model = SmilesLSTM(vocabulary, tk, model_parameters)
model.get_params()

{'dropout': 0.2,
 'layer_size': 5,
 'num_layers': 3,
 'cell_type': 'lstm',
 'embedding_layer_size': 5}

In [168]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and combined val/test sets
train_data, val_test_data = train_test_split(df, test_size=0.2, random_state=42)
# Splitting the combined val/test set into separate val and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.2, random_state=42)

train     = train_data.copy()
test      = test_data.copy()
valid     = val_data.copy()

# create a vocabulary using all SMILES in df
dataset = df['source'].unique().tolist() + df['target'].unique().tolist()
dataset = np.unique(dataset).tolist()

In [169]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_sequence(tokenizer_array, desired_length):
    padded_sequence = pad_sequences([tokenizer_array], maxlen=desired_length, padding='post')[0]
    return padded_sequence

In [170]:
X = train['source'].tolist()
y = train['target'].tolist()

def preprocess_smiles_data(X):
    for row in range(len(X)):
        X[row] = tk.tokenize(X[row])
        X[row] = vocabulary.encode(X[row] )
        X[row]  = pad_sequence(X[row], 200 )
        X[row]  = torch.tensor([X[row] ])
    return X

X = preprocess_smiles_data(X)
y = preprocess_smiles_data(y)

In [173]:
print(X_[0].shape)
print(numerical_indices_smi_sample.shape)

torch.Size([1, 200])
torch.Size([1, 3])


In [172]:
for epoch in range(3):
    
    model.zero_grad()
    out = model(numerical_indices_smi_sample)    

RuntimeError: input.size(-1) must be equal to input_size. Expected 86, got 5

In [ ]:
out

In [ ]:
class Trainer():
    def __init__(self, model: SmilesLSTM, input_smiles : Union[dict, str],
                 epochs : int=10, learning_rate : float=0.0001,
                 batch_size : int=250, shuffle : bool=True,
                 augment : int=0, output_model_path : str="./output/", start_epoch : int=0,
                 learning_rate_scheduler : str="StepLR", gamma : float=0.8,
                 eval_num_samples : int=64, eval_batch_size : int=64) -> None:
        
        # define the model
        self._model = model

        # define parameters
        self._batch_size        = batch_size
        self._learning_rate     = learning_rate
        self._epochs            = epochs
        self._start_epoch       = start_epoch
        self._output_model_path = output_model_path
        self._shuffle           = shuffle
        self._use_augmentation  = augment
        self._eval_num_samples  = eval_num_samples
        self._eval_batch_size   = eval_batch_size

        # define the data
        (self._train_dataloader,
         self._test_dataloader,
         self._valid_dataloader) = self._load_smiles(input_smiles=input_smiles)
        
        # define the optimizer and scheduler
        self._optimizer = torch.optim.Adam(params=self._model.network.parameters(),
                                           lr=self._learning_rate)
        
        # placeholders for the loss
        self._train_loss      = None
        self._valid_loss      = None
        self._best_valid_loss = None
        self._best_epoch      = None
        
        def _initialize_dataloader(self, smiles_list : list) -> toch.utils.data.Dataloader:
            dataset = Dataset(smiles_list=smiles_list,
                              vocabulary=self._model.vocabulary,
                              tokenizer=SMILESTokenizer())
            
            dataloader = torch.utils.data.DataLoader(dataset,
                                                     batch_size=self._batch_size,
                                                     shuffle=self._shuffle,
                                                     collate_fn=Dataset.collate_fn,
                                                     drop_last=True)
            return dataloader
        
        def _load_smiles(self, input_smiles : Union[dict, str]) -> Tuple[list, list, list]:
            # get values from dictionary
            train_smiles = input_smiles["train"]
            test_smiles  = input_smiles["test"]
            valid_smiles = input_smiles["valid"]

            # create the dataloader from the SMILES lists
            train_dataloader = self._initialize_dataloader(smiles_list=train_smiles)
            test_dataloader  = self._initialize_dataloader(smiles_list=test_smiles)
            valid_dataloader = self._initialize_dataloader(smiles_list=valid_smiles)
            
            return train_dataloader, test_dataloader, valid_dataloader      

# Train / validation / test split

In [ ]:
from sklearn.model_selection import train_test_split

print(df.shape)

# Splitting the data into train and combined val/test sets
train_data, val_test_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting the combined val/test set into separate val and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.2, random_state=42)

# Printing the sizes of the resulting splits
print("Train data size:", len(train_data))
print("Validation data size:", len(val_data))
print("Test data size:", len(test_data))

In [ ]:
train     = train_data.copy()
test      = test_data.copy()
valid     = val_data.copy()

# create a vocabulary using all SMILES in df
dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
dataset = np.unique(dataset).tolist()

tokenizer = SMILESTokenizer()
vocab     = create_vocabulary(smiles_list=dataset,
                                    tokenizer=tokenizer,
                                    canonical=False)

MAX_LENGTH = max(len(v) for v in dataset)

print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'Max length: {MAX_LENGTH}.\n')
# print(f'The unique tokens are: \n{vocabulary.tokens()}')

In [ ]:
tk = SMILESTokenizer()
vocab = Vocabulary()

smi_sample = 'CCBr'
tokenized_smi_sample = tk.tokenize(smi_sample, with_begin_and_end=False)
print(tokenized_smi_sample)
vocabulary.encode(tokenized_smi_sample)

### Function for pad sequencing

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_sequence(tokenizer_array, desired_length):
    padded_sequence = pad_sequences([tokenizer_array], maxlen=desired_length, padding='post')[0]
    return padded_sequence

In [ ]:
for d in [train, test, valid]:
    for c in d.columns:
        d[c] = d[c].apply(lambda x: tk.tokenize(x, with_begin_and_end=False))
        d[c] = d[c].apply(lambda x: vocabulary.encode(x).astype(int))
        d[c] = d[c].apply(lambda x: pad_sequence(x, MAX_LENGTH))

# Model Building

In [ ]:
# Convert the source and target columns into numpy arrays
trainX = np.array(train['source'].tolist())
trainY = np.array(train['target'].tolist())

print(trainX.shape, trainY.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

# Define the input shape
input_shape = (trainX.shape[1], 1)  # Assuming you want to feed one feature at a time

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=NUM_EMBEDDING, output_dim=EMBEDDING_DIM, input_length=MAX_LENGTH))
model.add(LSTM(units=128, input_shape=input_shape))
model.add(Dense(units=trainY.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trainX, trainY, epochs=3, batch_size=2048)

In [ ]:
# Convert the source and target columns into numpy arrays
testX = np.array(test['source'].tolist())
testY = np.array(test['target'].tolist())

print(testX.shape, testY.shape)

In [ ]:
pred = model.predict(testX)
pd.DataFrame(pred)

In [ ]:
pd.DataFrame(testY)